In [113]:
import numpy as np
from scipy.stats import unitary_group

np.set_printoptions(precision=3, suppress=True) #print less digits of floats for readability

In [204]:
def M(n, sigma, delta, m):
    r"""The Bell M matrix from Eq 1 of the paper"""
    mat = np.identity(m, dtype=np.complex128)
    mat[n, n] = np.exp(1j * sigma) * np.sin(delta)
    mat[n, n+1] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n+1] = -np.exp(1j * sigma) * np.sin(delta)
    return mat

def P(j, phi, m):
    mat = np.identity(m, dtype=np.complex128)
    mat[j,j] = np.exp(1j * phi)
    return mat

In [205]:
def is_unitary(U):
    return np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

# Reck

In [209]:
m = 7
# V = np.random.random((5,5)) + 1j*np.random.random((5,5))
U = unitary_group.rvs(m)

V = U.conj()

for j in range(m-1):
    x = m - 1
    y = j
    phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y])
    Pj = P(j, phi_j, m)
    V = V @ Pj
    for k in range(j+1):
        delta = np.arctan(-V[x,y+1] / V[x,y]).real
        V_temp = V @ M(j-k, 0, delta, m)
        sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
        n = j - k
        V = V @ M(n, sigma, delta, m)
        x -= 1
        y -= 1
    
# these next two lines are just to remove a global phase
zeta = - np.angle(V[0,0])
V = V @ P(0, zeta, m)

for j in range(1,m):
    zeta = np.angle(V[0,0]) - np.angle(V[j,j])
    V = V @ P(j, zeta, m)
        
V

array([[ 1.-0.j, -0.-0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.-0.j, -0.-0.j],
       [ 0.+0.j,  1.-0.j, -0.-0.j,  0.-0.j,  0.-0.j,  0.-0.j,  0.+0.j],
       [-0.-0.j,  0.-0.j,  1.-0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.-0.j],
       [ 0.+0.j, -0.-0.j,  0.+0.j,  1.-0.j, -0.-0.j, -0.+0.j,  0.-0.j],
       [-0.+0.j,  0.-0.j, -0.+0.j,  0.-0.j,  1.-0.j,  0.-0.j,  0.-0.j],
       [ 0.-0.j, -0.-0.j,  0.-0.j, -0.-0.j, -0.-0.j,  1.-0.j, -0.+0.j],
       [ 0.-0.j, -0.+0.j,  0.+0.j, -0.-0.j, -0.+0.j,  0.+0.j,  1.-0.j]])

# Clements

In [296]:
m = 4
# V = np.random.random((5,5)) + 1j*np.random.random((5,5))
U = unitary_group.rvs(m)

V = U.conj()

for j in range(m-1):
    #odd case in paper, because we index from 0 not 1
    if j % 2 == 0: 
        x = m - 1
        y = j 
        phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y]) # reversed order from paper
        V = V @ P(j, phi_j, m)
        for k in range(j+1):
            delta = np.arctan(-V[x,y+1] / V[x,y]) # flipped from paper
            n = j - k
            V_temp = V @ M(n, 0, delta, m)
            sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
            V = V @ M(n, sigma, delta, m)
            print(V)
            x -= 1
            y -= 1
    else:
        x = m - j - 1
        y = 0 
        phi_j = np.angle(V[x,y]) - np.angle(V[x-1,y])
        V = P(j, phi_j, m) @ V
        for k in range(j+1):
            delta = np.arctan(V[x-1,y] / V[x,y]) # flipped from paper
            V_temp = M(x-1, 0, delta, m) @ V
            n = m + k - j - 2
            if j != k:
                sigma = (np.angle(V_temp[x+1, y+1]) - np.angle(V_temp[x,y+1]))
            else:
                sigma = 0 
            V = M(n, sigma, delta, m) @ V
            print(V)
            x += 1
            y += 1
        
            

[[ 0.588-0.447j -0.092-0.033j  0.326-0.415j -0.405-0.048j]
 [ 0.38 +0.263j  0.076+0.369j -0.58 +0.188j -0.473+0.224j]
 [ 0.289-0.398j -0.201+0.072j -0.454+0.19j   0.403-0.556j]
 [-0.   -0.j     0.507-0.739j -0.322-0.067j -0.221-0.199j]]
[[ 0.588-0.447j -0.092-0.033j  0.326-0.415j -0.405-0.048j]
 [-0.07 -0.671j  0.084-0.067j  0.203+0.541j  0.275-0.352j]
 [-0.   -0.j     0.237-0.345j  0.524-0.073j  0.397+0.623j]
 [-0.   -0.j     0.507-0.739j -0.322-0.067j -0.221-0.199j]]
[[ 0.588-0.447j -0.092-0.033j  0.326-0.415j -0.405-0.048j]
 [-0.07 -0.671j  0.084-0.067j  0.203+0.541j  0.275-0.352j]
 [-0.   -0.j     0.56 -0.816j -0.07 -0.092j -0.032+0.083j]
 [-0.   -0.j    -0.   +0.j     0.61 -0.038j  0.453+0.648j]]
[[ 0.588-0.447j -0.092-0.033j  0.627+0.226j  0.   +0.j   ]
 [-0.07 -0.671j  0.084-0.067j -0.569+0.458j -0.   +0.j   ]
 [-0.   -0.j     0.56 -0.816j  0.082-0.12j   0.   -0.j   ]
 [-0.   -0.j    -0.   +0.j     0.   +0.j    -0.372-0.928j]]
[[ 0.588-0.447j -0.536+0.408j  0.   -0.j     0.   +0